In [24]:
import pygame as pg #for visuals
import time
import os
import random #for randomly changing height of the hurdles
pg.font.init()

In [25]:
w_width = 600
w_height = 750

#### Importing all the required images from the imgs folder to make a game frame.

In [26]:
planeimgs = [pg.image.load(os.path.join("imgs","planeYellow1.png")),
             pg.image.load(os.path.join("imgs","planeYellow2.png")),
             pg.image.load(os.path.join("imgs","planeYellow3.png"))] 
            #scale2x() makes the images twice the original size
            #load() loads the images

In [27]:
hurdleimg = pg.transform.scale2x(pg.image.load(os.path.join("imgs","rockGrass.png")))

In [28]:
baseimg = pg.transform.scale2x(pg.image.load(os.path.join("imgs","groundGrass.png")))

In [29]:
bgimg = pg.transform.scale2x(pg.image.load(os.path.join("imgs","background.png")))

stat_font=pg.font.SysFont("comicsans",50)

### Plane class

In [30]:
class Plane:
    maxrot = 25  #maximum rotation - how much the plane may tilt
    rotvel = 20  #how much to rotate each frame when the plane is rotated
    anitime = 5  #how long to show each plane animation
    imgg = planeimgs
    
    def __init__(self, x, y):
         
            #starting positions of the plane
        self.x = x
        self.y = y
        
        self.tilt = 0  #how much the plane is actually titled - 0 initially because we want it to start at a horizontal position
        self.tick_count = 0 #get to know how high or low the plane, when we last jumped
        self.vel = 0
        self.height = self.y
        self.img_count = 0 #so as to know which image is currently being displayed
        self.imgs = planeimgs[0] #basically first plane image
        
    def jump(self):
        self.vel = -10.5 #negative because in CG negative represents upward while positive represents downwards
        self.tick_count = 0 #set to 0 because we need to know when we are changing directions/velocities
        self.height = self.y #to keep track where the plane last jumped from
    
    def move(self): 
        self.tick_count += 1 #how many times we moved since the last jump
                #by moved, I mean the frame changed
            
        d = self.vel*(self.tick_count) + 1.5*(self.tick_count**2)  # calculate displacement
        
        #terminal velocity
        if d>=16:     #to not go less than 16px downwards
            d = (d/abs(d)) * 16
        
        if d<0 : 
            d -= 2  #to not jump abruptly
            
        self.y = self.y + d
        
        if d<0 or self.y<self.height+50 :  #if we are moving upwards 
            if self.tilt < self.maxrot: #so that we don't tilt the plane to some unexpected direction
                self.tilt = self.maxrot
                
        else:
            if self.tilt > -90:
                self.tilt -= self.rotvel  #rotating the plane completely downwards(90) 
    
    def draw(self, win):
        
        self.img_count += 1
        
        #deciding which plane image to show based on the time elapsed
        if self.img_count < self.anitime:
            self.imgs = planeimgs[0]
        elif self.img_count < self.anitime*2:
            self.imgs = planeimgs[1]
        elif self.img_count < self.anitime*3:
            self.imgs = planeimgs[2]
        elif self.img_count < self.anitime*4:
            self.imgs = planeimgs[1]
        elif self.img_count == self.anitime*4 + 1:
            self.imgs = planeimgs[0]
            self.img_count = 0
            
        # so when bird is nose diving it isn't flapping
        if self.tilt <= -80:
            self.imgs = planeimgs[1]
            self.img_count = self.anitime*2
            
        #rotating the plane
        rimg = pg.transform.rotate(self.imgs,self.tilt)
        n_rect = rimg.get_rect(center = self.imgs.get_rect(topleft = (self.x,self.y)).center)
        win.blit(rimg,n_rect.topleft)
        
    def get_mask(self):
        return pg.mask.from_surface(self.imgs)
    
    def xy(self):
        return type(self.imgs)

**Rock Class**

In [31]:
class Rock:
    gap=-100
    vel=5
    
    def __init__(self,x):
        self.x=x
        self.height=0
        self.top=0
        self.bottom=0
        self.rocktop=pg.transform.flip(hurdleimg,False,True) #flips for the upside dowm image
        self.rockbottom=hurdleimg
        self.passed= False #if the bird has already passed by the obstacle/collision purposes
        self.set_height()
    
    def set_height(self):
        self.height=random.randrange(200,400)
        self.top=self.height-self.rocktop.get_height()
        self.bottom=self.height-self.gap
    
    def move(self):
        self.x-=self.vel
    
    def draw(self,win):
        win.blit(self.rocktop,(self.x,self.top))
        win.blit(self.rockbottom,(self.x,self.bottom))
    
    def collide(self,plane):
        plane_mask=plane.get_mask()
        top_mask=pg.mask.from_surface(self.rocktop)
        bottom_mask=pg.mask.from_surface(self.rockbottom)
        
        #offsets represent how far these masks are from each other
        top_offset=(self.x-plane.x,self.top-round(plane.y)) #offset of the plane from the top mask
        bottom_offset=(self.x-plane.x,self.bottom-round(plane.y))
        
        #to find if the masks are colliding
        b_point=plane_mask.overlap(bottom_mask,bottom_offset)
        t_point=plane_mask.overlap(top_mask,top_offset)
        
        if t_point or b_point:
            return True #collision is occuring
        return False
    

**Ground Class**

In [32]:
class Ground:
    vel=5
    width=baseimg.get_width()
    img=baseimg

    def __init__(self,y):
        self.y=y
        self.x1=0
        self.x2=self.width
        
    def move(self):
        self.x1-=self.vel
        self.x2-=self.vel
        
        #for a continuously moving ground effect, we use two ground images that are displayed one after the other in a loop
        #if an image is off the screen, we cycle it back behind the next image and vice versa
        if self.x1+self.width<0 :
            self.x1= self.x2+self.width
        
        if self.x2+self.width<0 :
            self.x2=self.x1+self.width
            
    def draw(self,win):
        win.blit(self.img,(self.x1,self.y))
        win.blit(self.img,(self.x2,self.y))
        

In [33]:
def draw_window(win,plane,rocks,base,score):
    win.blit(bgimg, (0,0)) #blit just draws
    for rock in rocks:
        rock.draw(win)
    
    text= stat_font.render("Score: " + str(score),1,(255,255,255))
    #text= ("Score: " + str(score),1,(255,255,255))
    win.blit(text, (w_width - 10 - text.get_width(),10))
    
    base.draw(win)
    win.blit(baseimg, (0,625))
    plane.draw(win)
    pg.display.update()

In [34]:
plane = Plane(230,350)
base = Ground(730)
rocks=[Rock(713)]
win = pg.display.set_mode((w_width,w_height))
    
clock = pg.time.Clock()

score=0
    
run = True
    
while run:
        
    clock.tick(8) #the more the value here, the slower frames will change per second
    for event in pg.event.get():
        if event.type == pg.QUIT:
            run = False
                
    #plane.move() 
    add_rock=False
    rem=[]
    for rock in rocks:
        if rock.collide(plane):
            pass
        if rock.x+rock.rocktop.get_width()<0: #if the rock is completeley off the screen
            rem.append(rock)
        if not rock.passed and rock.x < plane.x:
            rock.passed=True
            add_rock=True 
        
        rock.move()
        
    if add_rock:
        score+=1
        rocks.append(Rock(713))
    
    for r in rem:
        rocks.remove(r)
    
    if plane.y+plane.imgs.get_height() >= 730:
        pass
        
    base.move()
    draw_window(win,plane,rocks,base,score)
            
pg.quit()
quit()   